# Concatenate boreholes

In [1]:
%matplotlib widget

In [2]:
from utils.io import genID_dated, gdf_viewer, gdf_geom
import re
import copy
import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import datetime
import matplotlib.pyplot as plt

## Extraction des dates de forages du type de forage et du foreur de l'étude de caractérisation

### Informations sur les forages de l'étude de caractérisation de 2010

In [3]:
Logs_forages = pd.read_excel('../../CF_data/Data_SITEREM/Logs forages.xls',sheet_name='Forages, tranchées')
gdf_viewer(Logs_forages)

Rows : 2016, columns : 16


interactive(children=(IntSlider(value=10, description='rows', max=2016, min=10, readout=False), IntSlider(valu…

In [4]:
Logs_forages = Logs_forages[['Date', 'N°', 'Id', 'Profondeur', 'Description', 'Piézo', 'Unnamed: 6',
                             'Gouge Ø75', 'MFT Ø145', 'carottier', 'tarrière', 'Liner Ø60']] 

In [5]:
Logs_forages.rename({'Gouge Ø75':'Gouge_75', 'Liner Ø60': 'Liner_60'}, axis=1, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [6]:
Logs_forages.loc[Logs_forages.query('Profondeur!=Profondeur' ).index,'Profondeur']=''

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [7]:
forages = Logs_forages.query('Profondeur.str.contains("Forage") and Profondeur!="Forage bloqué"', engine='python')
#Logs_forages["Id"].astype(str).str.contains('T')

In [8]:
forages['Ref'] = forages['Profondeur'].apply(lambda x: x.split('Forage ')[1].replace(' ', ''))

<ipython-input-8-07818ba03c75>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forages['Ref'] = forages['Profondeur'].apply(lambda x: x.split('Forage ')[1].replace(' ', ''))


In [9]:
forages['tmp'] = forages['Piézo'].replace(np.nan, '', regex=True) + '-' + forages['Unnamed: 6'].replace(np.nan, '', regex=True) + '-' + forages['Gouge_75'].replace(np.nan, '', regex=True)

<ipython-input-9-0d7b933e2cc3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forages['tmp'] = forages['Piézo'].replace(np.nan, '', regex=True) + '-' + forages['Unnamed: 6'].replace(np.nan, '', regex=True) + '-' + forages['Gouge_75'].replace(np.nan, '', regex=True)


In [10]:
#forages['Piezo'] = forages['tmp'].apply(lambda x: x.lstrip(' ').split('-')[0].lower()=='x') #1er code
forages['Type'] = forages['tmp'].apply(lambda x: 'Piezo' if x.lstrip(' ').split('-')[0].lower()=='x' else '')

<ipython-input-10-73fb0156ea3c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forages['Type'] = forages['tmp'].apply(lambda x: 'Piezo' if x.lstrip(' ').split('-')[0].lower()=='x' else '')


In [11]:
forages['Societe'] = forages['tmp'].apply(lambda x: x.rstrip('-').split('-')[-1].upper()) 

<ipython-input-11-1b9e989104a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forages['Societe'] = forages['tmp'].apply(lambda x: x.rstrip('-').split('-')[-1].upper())


In [12]:
forages.loc[123, 'Societe']='SITEREM'

In [13]:
forages = forages[['Date', 'Ref', 'Societe', 'Type']]
forages

,Date,Ref,Societe,Type
0,2010-03-11,1,SBS ENVIRONNEMENT,Piezo
9,2010-03-11,2,SBS ENVIRONNEMENT,
19,2010-03-11,3,SBS ENVIRONNEMENT,
27,2010-03-10,4,SBS ENVIRONNEMENT,
33,2010-03-12,5,SBS ENVIRONNEMENT,
...,...,...,...,...
1973,2010-08-03,612c,SBS ENVIRONNEMENT,
1977,2010-08-03,734,SITEREM,Piezo
1984,2010-09-01,FP595,SITEREM,Piezo
1994,2010-09-02,540,SITEREM,Piezo


In [14]:
str(forages['Date'][0].year)

'2010'

In [15]:
forages['ID'] = forages['Date'].apply(lambda x : str(x.year) + '-') + forages['Ref']
forages.drop('Ref', axis=1, inplace=True)

In [16]:
forages

,Date,Societe,Type,ID
0,2010-03-11,SBS ENVIRONNEMENT,Piezo,2010-1
9,2010-03-11,SBS ENVIRONNEMENT,,2010-2
19,2010-03-11,SBS ENVIRONNEMENT,,2010-3
27,2010-03-10,SBS ENVIRONNEMENT,,2010-4
33,2010-03-12,SBS ENVIRONNEMENT,,2010-5
...,...,...,...,...
1973,2010-08-03,SBS ENVIRONNEMENT,,2010-612c
1977,2010-08-03,SITEREM,Piezo,2010-734
1984,2010-09-01,SITEREM,Piezo,2010-FP595
1994,2010-09-02,SITEREM,Piezo,2010-540


In [17]:
forages[['ID', 'Type', 'Societe']].to_csv('../../CF_data/synthese/Result_traitem/boreholes_No_XYZ.csv', index=False) #No position

### Lecture des fichiers de position des forages du pilote (version 8)

In [18]:
v8 = gpd.read_file('../../CF_data/Data_UMONS/SIG/Pilote_v8.0.shp')
v8.query('GPS2021=="NON"')

,Id,Type,Pilote,Ref,diam,Valid,Syscal,Impo,Inox,Impo_perm,GPS2021,geometry
18,0,4,1,400,0,0,0,0,0,0,NON,POINT (152885.032 122585.367)
32,0,5,1,500,0,0,0,0,0,0,NON,POINT (152890.824 122583.580)
63,0,3,1,306,0,0,0,0,0,0,NON,POINT (152867.452 122588.877)


### forages phase 1

In [19]:
v2017 = gpd.read_file('../../CF_data/Data_UMONS/SIG/sondages_v2017-05-18.shp')
gdf_viewer(v2017, 10, 15)

Rows : 71, columns : 19


interactive(children=(IntSlider(value=10, description='rows', max=71, min=10, readout=False), IntSlider(value=…

In [20]:
v2017['Ref'] = v2017['Date'].apply(lambda x : str(datetime.datetime.strptime(x, '%Y-%m-%d').year) + '-') + v2017['Ref_sond']

### lecture des données des sondages antérieurs

In [21]:
sond_ant = pd.read_excel('../../CF_data/Data_UMONS/Logs_forages_vUmons_2018-03-20.xlsx', sheet_name='Sond_ant')

In [22]:
sond_ant['Ref'] = sond_ant['Ref_siterem'].apply(lambda x: '2010-' + str(x)) # Identifiant sur base de l'année

In [23]:
sond_ant

,Ref_GIS,Ref_siterem,X,Y,Type,Ref
0,1,500,152324.0,122975.0,Eau_RB,2010-500
1,2,501,152368.0,122909.0,Eau_RB,2010-501
2,3,511,152371.0,122771.0,Eau_RB,2010-511
3,4,513,152424.0,122752.0,Eau_RB,2010-513
4,5,514,152476.0,122786.0,Eau_RB,2010-514
...,...,...,...,...,...,...
252,253,520,152644.0,122791.0,SOL,2010-520
253,254,524,152570.0,122789.0,SOL,2010-524
254,255,525,152548.0,122783.0,SOL,2010-525
255,256,526,152553.0,122757.0,SOL,2010-526


In [24]:
sond_ant_gdf = gpd.GeoDataFrame(sond_ant, geometry=gpd.points_from_xy(sond_ant.X, sond_ant.Y, crs=str('EPSG:31370')))

### Lecture de l'extension du projet MEMORIS

In [25]:
ext = gpd.read_file('../../CF_data/Data_UMONS/SIG/Site_Memoris.shp')

### Affichage carte

In [26]:
def label(gdf, field, ax, size='xx-small'):
    for x, y, label in zip(gdf.geometry.x, gdf.geometry.y, gdf[field]):
        ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords='offset points', size=size)

In [27]:
fig, ax = plt.subplots(figsize=(6,6))
ext.plot(ax=ax, color='grey')
v2017.plot(ax=ax, color='r')

#for x, y, label in zip(v2017.geometry.x, v2017.geometry.y, v2017.Ref_sond):
#    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")
label(v2017, 'Ref', ax)
v8.plot(ax=ax, color='g')
label(v8, 'Ref', ax)  
sond_ant_gdf.plot(ax=ax, color='b', markersize=1)
label(sond_ant_gdf, 'Ref', ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

En ignorant Sond_ant_gdf

In [28]:
v8['new_Ref'] = 'F'+v8['Ref']

In [29]:
BH = []
BH.extend(v2017['Ref_sond'].to_list())
BH.extend(v8['new_Ref'].to_list())

In [30]:
v8['ID'] = v8.new_Ref
v2017['ID'] = v2017.Ref_sond

In [31]:
v8['Z']=101.

In [32]:
gdf = gpd.GeoDataFrame(pd.concat([v8[['geometry','ID', 'Z']], v2017[['geometry','ID', 'Z']]], ignore_index=True))

In [33]:
fig, ax = plt.subplots(figsize=(6,6))
ext.plot(ax=ax, color='lightgrey')
gdf.plot(ax=ax, markersize=1)
label(gdf, 'ID', ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
gdf['X']=gdf.geometry.x
gdf['Y']=gdf.geometry.y

In [35]:
gdf[['ID', 'X', 'Y', 'Z']].to_csv('../../CF_data/synthese/Result_traitem/boreholes_1.csv', index=False)

### Liste de tous les ouvrages références y compris les doublons entre jeux de données (et dans chaque jeu de donnée) 

In [36]:
BH = []
BH.extend(v2017['Ref_sond'].to_list())
BH.extend(v8['Ref'].to_list())
BH.extend(sond_ant_gdf['Ref_siterem'].to_list())

In [37]:
gdf = v2017.merge(sond_ant_gdf, how='outer', left_on='Ref_sond', right_on='Ref_siterem')
gdf[['Ref_sond', 'Ref_siterem']][0:100]

,Ref_sond,Ref_siterem
0,75,NaN
1,76,NaN
2,76b,76b
3,76c,76c
4,FP76,NaN
...,...,...
95,NaN,P1
96,NaN,P7
97,NaN,P8
98,NaN,P12


In [38]:
gdf.query('Ref_sond==Ref_siterem')['Ref_sond']

2      76b
3      76c
7      78b
9      P80
10     P81
16     97b
18     P99
24    P106
25    P107
29    538a
61     P22
65     P25
Name: Ref_sond, dtype: object

In [39]:
gdf.query('Ref_sond!=Ref_sond')['Ref_siterem'].to_list()

[500,
 501,
 511,
 513,
 514,
 517,
 521,
 522,
 523,
 528,
 529,
 530,
 531,
 531,
 532,
 '540sup',
 541,
 541,
 542,
 544,
 545,
 'FP115',
 'MW1',
 'MW2',
 'P1',
 'P7',
 'P8',
 'P12',
 'P15a',
 'P32',
 'P34',
 'P44',
 'P48b',
 'P51',
 'P58',
 'P59',
 'P64',
 'P65',
 'P67',
 'P74',
 'P86',
 'P91',
 'P109',
 'P110',
 'P112',
 'P125',
 'P143',
 'P152',
 'P162',
 'P164',
 'P165',
 'P168',
 'P170',
 'P175',
 'P181b',
 'P186',
 'P188',
 'P197',
 'P200',
 'P205',
 'P209',
 'P214',
 'P219b',
 'P400',
 'P405',
 'P406',
 'P410',
 'P411',
 'P595',
 '76sup',
 '63sup',
 '49sup',
 '14sup',
 'P512',
 504,
 509,
 510,
 533,
 534,
 536,
 537,
 539,
 540,
 '540a',
 543,
 595,
 734,
 '26275b',
 'FP14prof',
 'FP14sup',
 'P28',
 'P129',
 'P142',
 'P166',
 'P167',
 'P172',
 'P180',
 'P403',
 'P412',
 502,
 502,
 '160prof',
 '160sup',
 508,
 512,
 'FP15',
 'FP49prof',
 'FP49sup',
 'FP63prof',
 'FP63sup',
 'FP76prof',
 'FP76sup',
 'FP117prof',
 'FP600',
 'FP595',
 2,
 3,
 4,
 5,
 6,
 9,
 10,
 11,
 13,
 19,


In [40]:
len(BH)

416

In [41]:
len(set(BH)) # uniques values

395

In [42]:
gdf.query('Ref_siterem!=Ref_siterem')['Ref_sond'].to_list()

['75',
 '76',
 'FP76',
 '77',
 '78',
 '79',
 '82',
 '83',
 '95',
 '96',
 '97',
 '98',
 '100',
 '102',
 '103',
 '104',
 '105',
 '108',
 '536',
 '537',
 'P1aM',
 'P1bM',
 'P2M',
 'P3M',
 'P4M',
 'P5M',
 'P6M',
 'P7aM',
 'P7bM',
 'P8M',
 'P9aM',
 'P9bM',
 'P10M',
 'P11M',
 'P12M',
 'P13M',
 'P14M',
 'P15bM',
 'P15aM',
 "P15a'M",
 'P16M',
 'P17aM',
 'P17bM',
 'P17cM',
 'P17dM',
 'F18a',
 'F18b',
 'P18c',
 'P19',
 'P20',
 'P21',
 'P23',
 'F24a',
 'P24b',
 'P26',
 'F27a',
 'F27b',
 'F27c',
 'P27d']

In [43]:
gdf.columns

Index(['Ref_GIS_x', 'Ref_sond', 'XL72', 'YL72', 'Z', 'Refus', 'Date',
       'Prof_sond', 'Cote_fond', 'RB', 'ALL', 'S_A', 'S_S', 'Base_RB',
       'Cote_B_RB', 'Top_All', 'Top_SA', 'Top_SS', 'geometry_x', 'Ref_x', 'ID',
       'Ref_GIS_y', 'Ref_siterem', 'X', 'Y', 'Type', 'Ref_y', 'geometry_y'],
      dtype='object')

In [44]:
for idx, row in gdf.iterrows():
    if row['geometry_x'] is None:
        if row['geometry_y'] is not None:
            gdf.loc[idx, 'distance'] = 0.
        else:
            gdf.loc[idx, 'distance'] = None
    else:
        if row['geometry_y'] is None:
            gdf.loc[idx, 'distance'] = 0.
        else:
            gdf.loc[idx, 'distance'] = row['geometry_x'].distance(row['geometry_y'])

In [45]:
gdf.query('distance != distance') # recherche de forages n'ayant aucune coordonnées

,Ref_GIS_x,Ref_sond,XL72,YL72,Z,Refus,Date,Prof_sond,Cote_fond,RB,...,Ref_x,ID,Ref_GIS_y,Ref_siterem,X,Y,Type,Ref_y,geometry_y,distance


In [46]:
gdf.rename(columns={'Prof_sond':'Long'}, inplace=True)
gdf.loc[gdf.query('Refus==1').index, 'Refus']=True
gdf.loc[gdf.query('Refus==0').index, 'Refus']=False

In [47]:
gdf_viewer(gdf)

Rows : 316, columns : 29


interactive(children=(IntSlider(value=10, description='rows', max=316, min=10, readout=False), IntSlider(value…

In [48]:
pd.DataFrame(gdf)[['ID', 'X', 'Y', 'Z', 'Long','Refus']].to_csv('../../CF_data/synthese/Result_traitem/boreholes_1geom.csv', index=False)

//////////////////////////////////////--- Recupération de données des autres fichiers ---///////////////////////////////////////////////////////////////////////

## Lecture d'autres fichiers (1 ou plusieurs feuilles)

- Regrouper les infos sur les (ID, X, Y, Z, Long, Type) pour définir la géométrie des BH

- Regrouper les infos sur les (ID, X, Y, Z_top, Z_base, Type) pour définir les échantillons

- Regrouper les infos sur la lithologie (Z_top, Z_base) et les lier aux ID

- ...

### A - Fichiers contenant au moins les positions XYZ

### A.1- ../../CF_data//Data_UMONS/Logs_forages_vUmons_2018-03-20.xlsx
* **Feuille : 'Sond2017v3' (F)**

In [49]:
For_synth=gpd.GeoDataFrame({})

In [50]:
tmp_for = pd.read_excel('../../CF_data//Data_UMONS/Logs_forages_vUmons_2018-03-20.xlsx', sheet_name='Sond2017v3')#, skiprows=1)
gdf_viewer(tmp_for)

Rows : 89, columns : 32


interactive(children=(IntSlider(value=10, description='rows', max=89, min=10, readout=False), IntSlider(value=…

In [51]:
tmp_for.columns

Index(['Ref_GIS', 'Ref_sond', 'XL72', 'YL72', 'Z', 'Refus', 'Date',
       'Prof_sond', 'Cote_fond', 'RB', 'ALL', 'S_A', 'S_S', 'Base_RB',
       'Cote_B_RB', 'Top_All', 'Top_SA', 'Top_SS', 'Unnamed: 18',
       'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
       'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26',
       'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30',
       'Unnamed: 31'],
      dtype='object')

In [52]:
tmp_for.rename(columns={'Ref_sond':'Ref_x','XL72':'X', 'YL72':'Y', 'Z (m)':'Z',
        'Prof_sond':'Long', 'Base des Remblais (m)':'Base_RB',
        'Sommet des Alluvions (m)':'Top_All', 'Sommet du Socle altéré (m)':'Top_SA',
        'Sommet du Socle sain (m)':'Top_SS'}, inplace=True)

tmp_for.drop(columns=[tmp_for.columns.to_list()[x] for x in range(len(tmp_for.columns)) 
       if re.compile(r"Unnamed|Ref_GIS").match(tmp_for.columns.to_list()[x])], axis=1, inplace=True) 

tmp_for.columns

Index(['Ref_x', 'X', 'Y', 'Z', 'Refus', 'Date', 'Long', 'Cote_fond', 'RB',
       'ALL', 'S_A', 'S_S', 'Base_RB', 'Cote_B_RB', 'Top_All', 'Top_SA',
       'Top_SS'],
      dtype='object')

In [53]:
tmp_for=tmp_for[['Date', 'Ref_x', 'X', 'Y', 'Z', 'Long', 'Refus', 'Base_RB','Top_All', 'Top_SA','Top_SS',]]

In [54]:
tmp_for.loc[tmp_for.query('Long==9999').index, 'Long']=np.nan
tmp_for.loc[tmp_for.query('Z==9999').index, 'Z']=np.nan

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [55]:
for i,j in tmp_for.iterrows():
    if tmp_for.loc[i, 'Refus']==1:
        tmp_for.loc[i, 'Refus']=True
    else :
        tmp_for.loc[i, 'Refus']=False

In [56]:
geom = gpd.GeoSeries(tmp_for.apply(lambda x: Point(x['X'], x['Y']),1),crs={'init': 'epsg:31370'})
tmp_for = gpd.GeoDataFrame(tmp_for, geometry=geom, crs="EPSG:31370")
tmp_for

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,Date,Ref_x,X,Y,Z,Long,Refus,Base_RB,Top_All,Top_SA,Top_SS,geometry
0,2016-12-31,75,152836.000000,122609.000000,102.652,5.7,False,4.3,4.3,NaN,NaN,POINT (152836.000 122609.000)
1,2016-12-31,76,152867.000000,122611.000000,102.751,1.2,True,NaN,NaN,NaN,NaN,POINT (152867.000 122611.000)
2,2016-12-31,76b,152867.000000,122610.000000,102.751,3.0,True,NaN,NaN,NaN,NaN,POINT (152867.000 122610.000)
3,2016-12-31,76c,152867.000000,122608.000000,102.751,2.5,True,NaN,NaN,NaN,NaN,POINT (152867.000 122608.000)
4,2016-12-31,FP76,152860.000000,122608.000000,102.727,18.5,False,8.0,NaN,8.0,12.5,POINT (152860.000 122608.000)
...,...,...,...,...,...,...,...,...,...,...,...,...
84,NaT,F30,152887.892567,122581.184654,NaN,NaN,False,NaN,NaN,NaN,NaN,POINT (152887.893 122581.185)
85,NaT,F31,152885.916612,122578.766886,NaN,NaN,False,NaN,NaN,NaN,NaN,POINT (152885.917 122578.767)
86,NaT,F32,152885.673928,122581.757054,NaN,NaN,False,NaN,NaN,NaN,NaN,POINT (152885.674 122581.757)
87,NaT,F40,152883.685780,122577.833364,NaN,NaN,False,NaN,NaN,NaN,NaN,POINT (152883.686 122577.833)


In [57]:
len(set(tmp_for['Ref_x']))

89

In [58]:
For_synth=copy.deepcopy(tmp_for) #save tmp_for

In [59]:
#tmp_for = tmp_for.replace(np.nan, '', regex=True) #remove all 'NAN'
#pd.concat([v8[['geometry','ID', 'Z']], v2017[['geometry','ID', 'Z']]], ignore_index=True)

* **Feuille : 'DatabaseSOL20101005' (F,E)**

In [60]:
tmp_for = pd.read_excel('../../CF_data//Data_UMONS/Logs_forages_vUmons_2018-03-20.xlsx', sheet_name='DatabaseSOL20101005')#, skiprows=1)
gdf_viewer(tmp_for)

Rows : 758, columns : 26


interactive(children=(IntSlider(value=10, description='rows', max=758, min=10, readout=False), IntSlider(value…

In [61]:
tmp_for.columns

Index(['ID', 'CAMPAGNE', 'ZONE', 'N', 'AFFECTATIO', 'X', 'Y', 'Z',
       'PROF_FORAG', 'N_ECH', 'MIN_ECH', 'MAX_ECH', 'COUCHE', 'SOUMIS',
       'TERRAIN', 'DESCRIPTIO', 'INTENSITÉ', 'MIN_ORGANO', 'MAX_ORGANO', 'ML',
       'CN', 'BTEXS', 'HAP', 'EOX', 'HT', 'IPH'],
      dtype='object')

In [62]:
tmp_for=tmp_for[['N', 'X', 'Y', 'Z', 'PROF_FORAG', 'CAMPAGNE']]
tmp_for.rename(columns={'N':'Ref_y','PROF_FORAG':'Long', 'CAMPAGNE':'Societe'}, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [63]:
set(tmp_for['Societe'])

{'SBS', 'SITERE'}

In [64]:
for i,j in tmp_for.iterrows():
    if tmp_for.loc[i, 'Societe']=='SITERE':
        tmp_for.loc[i, 'Societe']='SITEREM'
    #else :
    #    tmp_for.loc[i, 'Refus']='-'

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [65]:
geom = gpd.GeoSeries(tmp_for.apply(lambda x: Point(x['X'], x['Y']),1),crs={'init': 'epsg:31370'})
tmp_for = gpd.GeoDataFrame(tmp_for, geometry=geom, crs="EPSG:31370")
tmp_for.head(5)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,Ref_y,X,Y,Z,Long,Societe,geometry
0,T220,152842.0,122770.0,NaN,1,SITEREM,POINT (152842.000 122770.000)
1,T219,152799.0,122808.0,NaN,1,SITEREM,POINT (152799.000 122808.000)
2,T218,152754.0,122758.0,NaN,2,SITEREM,POINT (152754.000 122758.000)
3,T218,152754.0,122758.0,NaN,2,SITEREM,POINT (152754.000 122758.000)
4,T217,152750.0,122713.0,NaN,3,SITEREM,POINT (152750.000 122713.000)


In [66]:
print('unique: ',len(set(tmp_for['Ref_y'])),'/', len(tmp_for))

unique:  233 / 758


In [67]:
tmp_for.drop_duplicates('Ref_y', inplace=True)
tmp_for.reset_index(inplace=True, drop=True)
tmp_for

,Ref_y,X,Y,Z,Long,Societe,geometry
0,T220,152842.0,122770.0,NaN,1,SITEREM,POINT (152842.000 122770.000)
1,T219,152799.0,122808.0,NaN,1,SITEREM,POINT (152799.000 122808.000)
2,T218,152754.0,122758.0,NaN,2,SITEREM,POINT (152754.000 122758.000)
3,T217,152750.0,122713.0,NaN,3,SITEREM,POINT (152750.000 122713.000)
4,T216,152702.0,122709.0,NaN,1,SITEREM,POINT (152702.000 122709.000)
...,...,...,...,...,...,...,...
228,5,153139.0,122614.0,"101,41",8,SBS,POINT (153139.000 122614.000)
229,4,153117.0,122614.0,"101,88",8,SBS,POINT (153117.000 122614.000)
230,3,153141.0,122640.0,"101,68",8,SBS,POINT (153141.000 122640.000)
231,2,153112.0,122638.0,"102,09",7,SBS,POINT (153112.000 122638.000)


In [68]:
For_synth

,Date,Ref_x,X,Y,Z,Long,Refus,Base_RB,Top_All,Top_SA,Top_SS,geometry
0,2016-12-31,75,152836.000000,122609.000000,102.652,5.7,False,4.3,4.3,NaN,NaN,POINT (152836.000 122609.000)
1,2016-12-31,76,152867.000000,122611.000000,102.751,1.2,True,NaN,NaN,NaN,NaN,POINT (152867.000 122611.000)
2,2016-12-31,76b,152867.000000,122610.000000,102.751,3.0,True,NaN,NaN,NaN,NaN,POINT (152867.000 122610.000)
3,2016-12-31,76c,152867.000000,122608.000000,102.751,2.5,True,NaN,NaN,NaN,NaN,POINT (152867.000 122608.000)
4,2016-12-31,FP76,152860.000000,122608.000000,102.727,18.5,False,8.0,NaN,8.0,12.5,POINT (152860.000 122608.000)
...,...,...,...,...,...,...,...,...,...,...,...,...
84,NaT,F30,152887.892567,122581.184654,NaN,NaN,False,NaN,NaN,NaN,NaN,POINT (152887.893 122581.185)
85,NaT,F31,152885.916612,122578.766886,NaN,NaN,False,NaN,NaN,NaN,NaN,POINT (152885.917 122578.767)
86,NaT,F32,152885.673928,122581.757054,NaN,NaN,False,NaN,NaN,NaN,NaN,POINT (152885.674 122581.757)
87,NaT,F40,152883.685780,122577.833364,NaN,NaN,False,NaN,NaN,NaN,NaN,POINT (152883.686 122577.833)


In [69]:
genID_dated(tmp_for, 'Ref_y', '2010'), genID_dated(For_synth, 'Ref_x')

Generation of ID-dated...
Using default date given !
Generation of ID-dated...
Using 'Date' column in the (geo)dataframe !


(0      2010-T220
 1      2010-T219
 2      2010-T218
 3      2010-T217
 4      2010-T216
          ...    
 228       2010-5
 229       2010-4
 230       2010-3
 231       2010-2
 232       2010-1
 Name: Ref_y, Length: 233, dtype: object,
 0       2016-75
 1       2016-76
 2      2016-76b
 3      2016-76c
 4     2016-FP76
         ...    
 84          F30
 85          F31
 86          F32
 87          F40
 88          F41
 Name: Ref_x, Length: 89, dtype: object)

In [70]:
For_synth=For_synth.append(tmp_for)

In [71]:
For_synth.reset_index(inplace=True, drop=True)
For_synth.loc[For_synth.query('Ref_x!=Ref_x').index, 'Ref_x']=For_synth.loc[For_synth.query('Ref_x!=Ref_x').index, 'Ref_y']
For_synth.drop('Ref_y', axis=1, inplace=True)

In [72]:
For_synth.loc[For_synth.query("Date >= '2017-02-22' and Date <= '2017-05-18'").index, "Societe"]='ECOPLANNING'
For_synth.rename(columns={'Ref_x':'ID'}, inplace=True)
#df.query()'Profondeur.str.contains("Forage") and Profondeur!="Forage bloqué"', engine='python')
#df.loc[(df['date'] >= '2020-09-01') & (df['date'] < '2020-09-15')] 
#df.query("date >= '2020-08-01' and date < '2020-09-01'") 

In [73]:
gdf_viewer(For_synth)

Rows : 322, columns : 13


interactive(children=(IntSlider(value=10, description='rows', max=322, min=10, readout=False), IntSlider(value…

In [74]:
pd.DataFrame(For_synth)[['ID', 'X', 'Y', 'Z', 'Long', 'Refus', 'Societe']].to_csv('../../CF_data/synthese/Result_traitem/boreholes_2.csv', index=False)

* **Feuilles : 'Sond_all_loc' (F) / 'Feuil1' {autre fichier}(F,E) / 'Obs_siterem_phase2' (F)**

In [75]:
tmp_for = pd.read_excel('../../CF_data//Data_UMONS/Logs_forages_vUmons_2018-03-20.xlsx', sheet_name='Sond_all_loc')#, skiprows=1)
gdf_viewer(tmp_for)

Rows : 89, columns : 6


interactive(children=(IntSlider(value=10, description='rows', max=89, min=10, readout=False), IntSlider(value=…

In [76]:
tmp_for.rename(columns={'Ref_sond':'Ref_x','XL72':'X', 'YL72':'Y'}, inplace=True)
tmp_for=tmp_for[['Ref_x', 'X', 'Y', 'Z']]
len(tmp_for)

89

In [77]:
gdf_geom(tmp_for)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,Ref_x,X,Y,Z,geometry
0,75,152836.0,122609.0,102.652,POINT (152836.000 122609.000)
1,76,152867.0,122611.0,102.751,POINT (152867.000 122611.000)
2,77,152898.0,122606.0,102.837,POINT (152898.000 122606.000)
3,78,152864.0,122589.0,102.709,POINT (152864.000 122589.000)
4,79,152893.0,122592.0,102.690,POINT (152893.000 122592.000)


In [78]:
For_synth=copy.deepcopy(tmp_for) #save tmp_for

In [79]:
tmp_for = pd.read_excel('../../CF_data//Data_UMONS/ouvrages/profondeur de contact campagne de forages octobre 2019.xlsx', sheet_name='Feuil1', skiprows=2)
gdf_viewer(tmp_for)

Rows : 10, columns : 5


interactive(children=(IntSlider(value=10, description='rows', max=10, min=10, readout=False), IntSlider(value=…

In [80]:
tmp_for.rename(columns={'n°forage ':'Ref_y','x':'X', 'y':'Y', 'z':'Z', 'profondeur(m)':'Long'}, inplace=True)
tmp_for=tmp_for[['Ref_y', 'X', 'Y', 'Z', 'Long']]

In [81]:
tmp_for=tmp_for[:8]

In [82]:
gdf_geom(tmp_for)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,Ref_y,X,Y,Z,Long,geometry
0,205.0,152887.693,122594.620,101.804,3.2,POINT (152887.693 122594.620)
1,208.0,152885.296,122592.986,101.848,3.4,POINT (152885.296 122592.986)
2,212.0,152882.850,122591.453,101.930,3.4,POINT (152882.850 122591.453)
3,207.0,152892.925,122592.662,101.889,3.4,POINT (152892.925 122592.662)
4,214.0,152888.082,122588.486,101.854,3.6,POINT (152888.082 122588.486)


In [83]:
For_synth.merge(tmp_for, how='inner', left_on='Ref_x', right_on='Ref_y')

,Ref_x,X_x,Y_x,Z_x,geometry_x,Ref_y,X_y,Y_y,Z_y,Long,geometry_y


In [84]:
genID_dated(tmp_for, 'Ref_y', '2019')

Generation of ID-dated...
Using default date given !


0    2019-205.0
1    2019-208.0
2    2019-212.0
3    2019-207.0
4    2019-214.0
5    2019-217.0
6    2019-225.0
7    2019-304.0
Name: Ref_y, dtype: object

In [85]:
For_synth=For_synth.append(tmp_for)

In [86]:
For_synth.reset_index(inplace=True, drop=True)
For_synth.loc[For_synth.query('Ref_x!=Ref_x').index, 'Ref_x']=For_synth.loc[For_synth.query('Ref_x!=Ref_x').index, 'Ref_y']
For_synth.drop('Ref_y', axis=1, inplace=True)

In [87]:
gdf_viewer(For_synth)

Rows : 97, columns : 6


interactive(children=(IntSlider(value=10, description='rows', max=97, min=10, readout=False), IntSlider(value=…

In [88]:
tmp_for = pd.read_excel('../../CF_data//Data_UMONS/Logs_forages_vUmons_2018-03-20.xlsx', sheet_name='Obs_siterem_phase2')#, skiprows=2)
gdf_viewer(tmp_for)

Rows : 22, columns : 13


interactive(children=(IntSlider(value=10, description='rows', max=22, min=10, readout=False), IntSlider(value=…

In [89]:
tmp_for.rename(columns={'Date':'Date_Prv','ref_piezo':'Ref_Pz', 'Prof_piézo':'Long_Pz', 'Nappe':'Aquifer', 'Niv_nappe':'Niv_Pz'}, inplace=True)
tmp_for=tmp_for[['Date_Prv','Ref_Pz', 'X', 'Y', 'Z', 'Long_Pz', 'Niv_Pz', 'Aquifer']]
tmp_for['Type']=tmp_for['Long_Pz'].apply(lambda x: 'Piezo' if not pd.isnull(x) else '')

<ipython-input-89-a816405946c6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_for['Type']=tmp_for['Long_Pz'].apply(lambda x: 'Piezo' if not pd.isnull(x) else '')


In [90]:
tmp_for.columns

Index(['Date_Prv', 'Ref_Pz', 'X', 'Y', 'Z', 'Long_Pz', 'Niv_Pz', 'Aquifer',
       'Type'],
      dtype='object')

In [91]:
#merge_tmp=For_synth.merge(tmp_for, how='inner', left_on='Ref_x', right_on='Ref_Pz', indicator=True)
#gdf_viewer(merge_tmp[['Ref_x', 'Ref_Pz','geometry_x', 'geometry_y']])

In [92]:
merge_tmp=For_synth.merge(tmp_for, how='outer', left_on='Ref_x', right_on='Ref_Pz')
merge_tmp.loc[merge_tmp.query('Ref_x==Ref_Pz').index, 'Long']=merge_tmp['Long_Pz']
merge_tmp=merge_tmp[['Date_Prv', 'Ref_x', 'X_x', 'Y_x', 'Z_x', 'Long', 'Niv_Pz', 'Type', 'Aquifer']]

In [93]:
#gdf_viewer(merge_tmp.query('Ref_x==Ref_Pz'))
gdf_viewer(merge_tmp)

Rows : 97, columns : 9


interactive(children=(IntSlider(value=10, description='rows', max=97, min=10, readout=False), IntSlider(value=…

In [94]:
len(set(merge_tmp['Ref_x']))

97

In [95]:
merge_tmp.rename(columns={'Ref_x':'ID', 'X_x':'X', 'Y_x':'Y', 'Z_x':'Z'}, inplace=True)

In [96]:
merge_tmp[['ID','X','Y','Z','Long','Type']].to_csv('../../CF_data/synthese/Result_traitem/boreholes_3.csv', index=False)

* **Feuille : 'Analyse_eau_Phases1&2_toSIG' (F,E)**

In [97]:
tmp_for = pd.read_excel('../../CF_data//Data_UMONS/Logs_forages_vUmons_2018-03-20.xlsx', sheet_name='Analyse_eau_Phases1&2_toSIG')#, skiprows=2)
gdf_viewer(tmp_for)

Rows : 45, columns : 87


interactive(children=(IntSlider(value=10, description='rows', max=45, min=10, readout=False), IntSlider(value=…

In [98]:
# ici c'est la date de prélèvement des échantillons d'eau !!!
tmp_for.rename(columns={'Crep_min':'Crep_top','Crep_max':'Crep_base','Date':'Date_Prv','Ref_forage':'ID',
                        'XL72':'X', 'YL72':'Y','Prof_forage':'Long','Prof_ piezo':'Long_Pz'}, inplace=True)
tmp_for.loc[tmp_for.query('Crep_top==999').index, 'Crep_top']=np.nan
tmp_for.loc[tmp_for.query('Crep_base==999').index, 'Crep_base']=np.nan
tmp_for.loc[tmp_for.query('Long_Pz==9999.0').index, 'Long_Pz']=np.nan

tmp_for['Type']=tmp_for['Crep_top'].apply(lambda x: 'Piezo' if not pd.isnull(x) else '')
#tmp_for['Date_Prv'] = tmp_for['Date_Prv'].apply(lambda x : datetime.date(1900, 1, 1) + datetime.timedelta(days=x-2))
tmp_for=tmp_for[['Date_Prv','ID', 'X', 'Y','Z', 'Long', 'Type', 'Long_Pz','Crep_top', 'Crep_base']]
gdf_viewer(tmp_for, un_val='ID')

Rows : 45, columns : 10, Unique on 'ID': 30


interactive(children=(IntSlider(value=10, description='rows', max=45, min=10, readout=False), IntSlider(value=…

In [99]:
tmp_for.drop_duplicates('ID', inplace=True)
tmp_for.reset_index(inplace=True, drop=True)

<ipython-input-99-fc812858cef0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_for.drop_duplicates('ID', inplace=True)


In [100]:
For_synth=copy.deepcopy(tmp_for)

* **Feuille : 'Analyse_Sol_Phases1&2' (F,E)**

In [101]:
tmp_for = pd.read_excel('../../CF_data//Data_UMONS/Logs_forages_vUmons_2018-03-20.xlsx', sheet_name='Analyse_sol_Phases1&2', skiprows=4)
gdf_viewer(tmp_for)

Rows : 60, columns : 84


interactive(children=(IntSlider(value=10, description='rows', max=60, min=10, readout=False), IntSlider(value=…

In [102]:
# ici c'est la date de prélèvement des échantillons d'eau !!!
tmp_for.rename(columns={'Date':'Date_Prv','Ref_forage':'Ref_x','XL72':'X', 'YL72':'Y','Unnamed: 8':'Long',
                        'Unnamed: 9':'Refus'}, inplace=True)
#tmp_for['Type']='Piezo'
#tmp_for['Date_Prv'] = tmp_for['Date_Prv'].apply(lambda x : datetime.date(1900, 1, 1) + datetime.timedelta(days=x-2))
tmp_for=tmp_for[['Date_Prv','Ref_x', 'X', 'Y','Z', 'Long', 'Refus']]
gdf_viewer(tmp_for)

Rows : 60, columns : 7


interactive(children=(IntSlider(value=10, description='rows', max=60, min=10, readout=False), IntSlider(value=…

In [103]:
len(set(tmp_for['Ref_x']))

33

In [104]:
tmp_for.drop_duplicates('Ref_x', inplace=True)
tmp_for.reset_index(inplace=True, drop=True)

<ipython-input-104-5a18f9753ff3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_for.drop_duplicates('Ref_x', inplace=True)


In [105]:
tmp_for['Refus'] = tmp_for['Refus'].apply(lambda x: True if x.lstrip(' ').split('-')[0].lower()=='x' else False)
tmp_for.loc[tmp_for.query('Z==9999').index, 'Z']=np.nan
tmp_for.loc[[31,32], 'Refus']=True

<ipython-input-105-d29e5e0a0233>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_for['Refus'] = tmp_for['Refus'].apply(lambda x: True if x.lstrip(' ').split('-')[0].lower()=='x' else False)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [106]:
merge_tmp=For_synth.merge(tmp_for, how='outer', left_on='ID', right_on='Ref_x')

In [107]:
for idx, row in merge_tmp.iterrows():
    if pd.isnull(merge_tmp.loc[idx, 'ID']) is True:
        #merge_tmp.loc[idx, ['ID','X_x', 'Y_x', 'Z_x', 'Long_x', 'Long_Pz']]=merge_tmp.loc[idx, ['Ref_x','X_y', 'Y_y', 'Z_y', 'Long_y']]
        merge_tmp.loc[idx, 'ID']=merge_tmp.loc[idx, 'Ref_x']
        merge_tmp.loc[idx, 'X_x']=merge_tmp.loc[idx, 'X_y']
        merge_tmp.loc[idx, 'Y_x']=merge_tmp.loc[idx, 'Y_y']
        merge_tmp.loc[idx, 'Z_x']=merge_tmp.loc[idx, 'Z_y']
        merge_tmp.loc[idx, 'Long_x']=merge_tmp.loc[idx, 'Long_y']

merge_tmp=merge_tmp[['ID', 'X_x', 'Y_x', 'Z_x', 'Long_x', 'Long_Pz', 'Type', 'Crep_top','Crep_base','Refus']]
merge_tmp.rename(columns={'X_x':'X', 'Y_x':'Y', 'Z_x':'Z', 'Long_x':'Long'}, inplace=True)

In [108]:
#gdf_viewer(merge_tmp.query('ID!=Ref_x')) #correspond à (query 'how=inner')!
#gdf_viewer(merge_tmp[['Refus','Long_Pz']])
gdf_viewer(merge_tmp, un_val='ID')

Rows : 38, columns : 10, Unique on 'ID': 38


interactive(children=(IntSlider(value=10, description='rows', max=38, min=10, readout=False), IntSlider(value=…

In [109]:
merge_tmp.to_csv('../../CF_data/synthese/Result_traitem/boreholes_4.csv', index=False)

### A.2- ../Data_UMONS/ouvrages/Database_piezometres_et_forages.xlsx
* **Feuille : 'DONNEES PIEZOS' (F)**

In [110]:
For_synth=gpd.GeoDataFrame({})

In [111]:
tmp_for = pd.read_excel('../../CF_data/Data_UMONS/ouvrages/Database_piezometres_et_forages.xlsx', sheet_name='DONNEES PIEZOS', skiprows=2)
gdf_viewer(tmp_for)

Rows : 147, columns : 185


interactive(children=(IntSlider(value=10, description='rows', max=147, min=10, readout=False), IntSlider(value…

In [112]:
tmp_for.rename(columns={'N_piezo.':'Ref_Pz','X [m]':'X','Y [m]':'Y','Z tête PZ [m]':'Z','Zsol [m]':'Zsol', 
                        'Prof_PZ [m]':'Long_Pz','Section_crépinée [m]':'Long_Crep','Diamètre_int [m]':'Diam_Pz'}, inplace=True)

tmp_for=tmp_for[['Ref_Pz','X', 'Y', 'Z', 'Zsol', 'Long_Pz', 'Long_Crep', 'Diam_Pz']][:130]
tmp_for.replace('-',np.nan, inplace=True)
tmp_for.rename(columns={'Ref_Pz':'ID'}, inplace=True)

In [113]:
tmp_for['Type']=tmp_for['Long_Crep'].apply(lambda x: 'Piezo' if not pd.isnull(x) else '')

In [114]:
gdf_viewer(tmp_for) # all units in [m], no dates

Rows : 130, columns : 9


interactive(children=(IntSlider(value=10, description='rows', max=130, min=10, readout=False), IntSlider(value…

In [115]:
For_synth=copy.deepcopy(tmp_for)

### A.3- ../Data_UMONS/ouvrages/Profils sols et données forages.xls
* **Feuille : 'Données de forage' (F)**

In [116]:
tmp_for = pd.read_excel('../../CF_data/Data_UMONS/ouvrages/Profils sols et données forages.xls', sheet_name='Données de forage')#, skiprows=2)
gdf_viewer(tmp_for)

Rows : 25, columns : 15


interactive(children=(IntSlider(value=10, description='rows', max=25, min=10, readout=False), IntSlider(value=…

In [117]:
tmp_for.rename(columns={'Profondeur':'Long', 'Méthode':'Method', 'Diamètre forage':'Diam','Niv. Eau p/r sol':'Niv_Pz_sol',
                        'PZ Prof.':'Long_Pz', 'PZ Diamètre':'Diam_Pz','PZ L.crépinée':'Long_Crep', 'Société forage':'Societe'}, inplace=True)
tmp_for=tmp_for[['ID', 'X', 'Y', 'Z', 'Date', 'Long', 'Method', 'Diam', 'Remarque','Niv_Pz_sol', 
                 'Long_Pz', 'Diam_Pz', 'Long_Crep', 'Societe']]

In [118]:
tmp_for['Type'] = tmp_for['Long_Pz'].apply(lambda x: '' if pd.isnull(x) else 'Piezo')
tmp_for['Refus'] = tmp_for['Remarque'].apply(lambda x: False if pd.isnull(x) else True)
tmp_for.drop('Remarque', axis=1, inplace=True)

In [119]:
gdf_viewer(tmp_for)

Rows : 25, columns : 15


interactive(children=(IntSlider(value=10, description='rows', max=25, min=10, readout=False), IntSlider(value=…

In [120]:
For_synth=For_synth.append(tmp_for)

In [121]:
For_synth.reset_index(inplace=True, drop=True)
genID_dated(For_synth, 'ID')

Generation of ID-dated...
Using 'Date' column in the (geo)dataframe !


0               1
1              12
2              22
3              25
4              32
          ...    
150     2017-F16M
151    2017-F17aM
152    2017-F17bM
153    2017-F17cM
154    2017-F17dM
Name: ID, Length: 155, dtype: object

In [122]:
gdf_viewer(For_synth)

Rows : 155, columns : 16


interactive(children=(IntSlider(value=10, description='rows', max=155, min=10, readout=False), IntSlider(value…

In [123]:
For_synth.to_csv('../../CF_data/synthese/Result_traitem/boreholes_5.csv', index=False)

* **Feuille : 'Piézométrie' (F)**

In [124]:
tmp_for = pd.read_excel('../../CF_data/Data_UMONS/ouvrages/Profils sols et données forages.xls', sheet_name='Piézométrie', skiprows=1)
gdf_viewer(tmp_for)

Rows : 37, columns : 21


interactive(children=(IntSlider(value=10, description='rows', max=37, min=10, readout=False), IntSlider(value=…

In [125]:
merge_tmp=copy.deepcopy(tmp_for[:11])
merge_tmp.reset_index(inplace=True, drop=True)
merge_tmp.rename(columns={'z':'Z',}, inplace=True)
merge_tmp=merge_tmp[['ID', 'Z']]
merge_tmp['Type']='Piezo'

gdf_viewer(merge_tmp, un_val='ID')

Rows : 11, columns : 3, Unique on 'ID': 11


interactive(children=(IntSlider(value=10, description='rows', max=11, min=10, readout=False), IntSlider(value=…

In [126]:
tmp_for=tmp_for[16:]
tmp_for.reset_index(inplace=True, drop=True)
tmp_for.drop('ID', axis=1, inplace=True)
tmp_for.rename(columns={'Unnamed: 8':'Date_Prv','Unnamed: 9':'Terrain', 'Unnamed: 10':'ID'}, inplace=True)
tmp_for=tmp_for[['ID']]
tmp_for['Type']='Piezo'
gdf_viewer(tmp_for)

Rows : 21, columns : 2


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


interactive(children=(IntSlider(value=10, description='rows', max=21, min=10, readout=False), IntSlider(value=…

In [127]:
merge_tmp.merge(tmp_for, how='inner', left_on='ID', right_on='ID')

,ID,Z,Type_x,Type_y


In [128]:
merge_tmp=merge_tmp.append(tmp_for)

In [129]:
merge_tmp.reset_index(inplace=True, drop=True)
merge_tmp.drop(merge_tmp.query("ID!=ID").index, inplace=True) # supprimer les lignes avec ID='NaN'
merge_tmp.reset_index(inplace=True, drop=True)
gdf_viewer(merge_tmp)

Rows : 28, columns : 3


interactive(children=(IntSlider(value=10, description='rows', max=28, min=10, readout=False), IntSlider(value=…

In [130]:
For_synth=copy.deepcopy(merge_tmp)

* **Feuille : 'Equipement' (F)**

In [131]:
tmp_for = pd.read_excel('../../CF_data/Data_UMONS/ouvrages/Profils sols et données forages.xls', sheet_name='Equipement')#, skiprows=1)
gdf_viewer(tmp_for)

Rows : 36, columns : 7


interactive(children=(IntSlider(value=10, description='rows', max=36, min=10, readout=False), IntSlider(value=…

In [132]:
tmp_for.rename(columns={'Diam. Forage':'Diam', 'Diamètre ext. PZ':'Diam_Pz'}, inplace=True)
tmp_for[['Crep_top','Crep_base','Bent_top','Bent_base']]=np.nan

In [133]:
for idx, row in tmp_for.iterrows():
    if row['Légende']=='Bentonite':
        tmp_for.loc[idx,'Bent_top']=row['De']
        tmp_for.loc[idx,'Bent_base']=row['A']
    elif row['Légende']=="Crépine":
        tmp_for.loc[idx,'Crep_top']=row['De']
        tmp_for.loc[idx,'Crep_base']=row['A']

In [134]:
i=0
for idx in range(len(tmp_for)-1):
    i+=1
    if tmp_for.loc[idx,'ID']==tmp_for.loc[i,'ID'] and pd.isnull(tmp_for.loc[idx,'Crep_top']) and not pd.isnull(tmp_for.loc[i,'Crep_top']):
        tmp_for.loc[idx,'Crep_top']=tmp_for.loc[i,'Crep_top']
        tmp_for.loc[idx,'Crep_base']=tmp_for.loc[i,'Crep_base']
        
    elif tmp_for.loc[idx,'ID']==tmp_for.loc[i,'ID'] and pd.isnull(tmp_for.loc[idx,'Bent_top']) and not pd.isnull(tmp_for.loc[i,'Bent_top']):
        tmp_for.loc[idx,'Bent_top']=tmp_for.loc[i,'Bent_top']
        tmp_for.loc[idx,'Bent_base']=tmp_for.loc[i,'Bent_base']

In [135]:
tmp_for=tmp_for[['ID','Diam','Diam_Pz','Crep_top','Crep_base','Bent_top','Bent_base']]
tmp_for.drop_duplicates('ID', inplace=True)
tmp_for.reset_index(inplace=True, drop=True)
tmp_for['Type']='Piezo'

<ipython-input-135-52f7b3a3f1d0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_for.drop_duplicates('ID', inplace=True)
<ipython-input-135-52f7b3a3f1d0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_for['Type']='Piezo'


In [136]:
For_synth=For_synth.merge(tmp_for, how='outer', left_on='ID', right_on='ID')
For_synth.rename(columns={'Type_x':'Type'}, inplace=True)
For_synth.drop('Type_y', axis=1, inplace=True)

gdf_viewer(For_synth, un_val='ID')

Rows : 29, columns : 9, Unique on 'ID': 29


interactive(children=(IntSlider(value=10, description='rows', max=29, min=10, readout=False), IntSlider(value=…

In [137]:
For_synth.to_csv('../../CF_data/synthese/Result_traitem/boreholes_6.csv', index=False)

### A.4- ../Data_UMONS/Contamination/Doc_SITEREM/Rapport de synthèse des études de sol et des eaux souterraines. Siterem - 2017/Database MEMORIS3.xlsx
* **Feuille : 'DONNEES PIEZOS' (F)**

In [138]:
For_synth=gpd.GeoDataFrame({})

In [139]:
tmp_for = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Doc_SITEREM/Rapport de synthèse des études de sol et des eaux souterraines. Siterem - 2017/Database MEMORIS3.xlsx', sheet_name='DONNEES PIEZOS', skiprows=2)
gdf_viewer(tmp_for)

Rows : 147, columns : 186


interactive(children=(IntSlider(value=10, description='rows', max=147, min=10, readout=False), IntSlider(value…

In [140]:
tmp_for.rename(columns={'Campagne':'Societe','N_piezo.':'Ref_Pz','X [m]':'X','Y [m]':'Y','Z tête PZ [m]':'Z','Zsol [m]':'Zsol', 
                        'Prof_PZ [m]':'Long_Pz','Section_crépinée [m]':'Long_Crep','Diamètre_int [m]':'Diam_Pz'}, inplace=True)

tmp_for=tmp_for[['Ref_Pz','X', 'Y', 'Z', 'Zsol', 'Long_Pz', 'Long_Crep', 'Diam_Pz', 'Societe']][:130]
tmp_for.rename(columns={'Ref_Pz':'ID'}, inplace=True)
tmp_for.replace('-',np.nan, inplace=True)

In [141]:
tmp_for['Type']=tmp_for['Long_Crep'].apply(lambda x: 'Piezo' if not pd.isnull(x) else '')

In [142]:
gdf_viewer(tmp_for, un_val='ID') # all units in [m]

Rows : 130, columns : 10, Unique on 'ID': 130


interactive(children=(IntSlider(value=10, description='rows', max=130, min=10, readout=False), IntSlider(value…

In [143]:
tmp_for.to_csv('../../CF_data/synthese/Result_traitem/boreholes_7.csv', index=False)

* **Feuille : 'DRAINS ET PIEZOS ENEL' (F)**

In [144]:
tmp_for = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Doc_SITEREM/Rapport de synthèse des études de sol et des eaux souterraines. Siterem - 2017/Database MEMORIS3.xlsx', sheet_name='DRAINS ET PIEZOS ENEL', skiprows=1)
gdf_viewer(tmp_for)

Rows : 147, columns : 68


interactive(children=(IntSlider(value=10, description='rows', max=147, min=10, readout=False), IntSlider(value…

In [145]:
tmp_for.rename(columns={'N°':'ID', 'Date ':'Date','Hauteur de la chambre ':'Ht_Chambre'}, inplace=True)
tmp_for=tmp_for[['ID', 'Date', 'X', 'Y', 'Zsol', 'Ht_Chambre', 'ETUDE']][2:20]
tmp_for.replace('-',np.nan, inplace=True)
#tmp_for['Zsol']=tmp_for['Zsol'].apply(lambda x: x if not pd.isnull(x) else np.nan)

gdf_viewer(tmp_for, un_val='ID')

Rows : 18, columns : 7, Unique on 'ID': 18


interactive(children=(IntSlider(value=10, description='rows', max=18, min=10, readout=False), IntSlider(value=…

In [146]:
For_synth=copy.deepcopy(tmp_for)

* **Feuille : 'RESULTS_EAU' (F)**

In [147]:
tmp_for = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Doc_SITEREM/Rapport de synthèse des études de sol et des eaux souterraines. Siterem - 2017/Database MEMORIS3.xlsx', sheet_name='RESULTS_EAU', skiprows=1)
gdf_viewer(tmp_for)

Rows : 204, columns : 185


interactive(children=(IntSlider(value=10, description='rows', max=204, min=10, readout=False), IntSlider(value…

In [148]:
tmp_for.rename(columns={'Campagne':'Societe','N_piezo.':'ID','Z tête PZ':'Z','Prof_PZ':'Long_Pz',
                        'Section_crépinée':'Long_Crep','Diamètre_int':'Diam_Pz'}, inplace=True)

tmp_for=tmp_for[['ID','X','Y','Z','Zsol','Long_Pz','Long_Crep','Diam_Pz','Societe']][1:130]

tmp_for.replace('-',np.nan, inplace=True)

In [149]:
tmp_for['Type']=tmp_for['Long_Crep'].apply(lambda x: 'Piezo' if not pd.isnull(x) else '')

In [150]:
gdf_viewer(tmp_for, un_val='ID') # all units in [m]

Rows : 129, columns : 10, Unique on 'ID': 129


interactive(children=(IntSlider(value=10, description='rows', max=129, min=10, readout=False), IntSlider(value…

In [151]:
For_synth.merge(tmp_for, how='inner', left_on='ID', right_on='ID')

,ID,Date,X_x,Y_x,Zsol_x,Ht_Chambre,ETUDE,X_y,Y_y,Z,Zsol_y,Long_Pz,Long_Crep,Diam_Pz,Societe,Type


In [152]:
tmp_for=tmp_for.append(For_synth)

In [153]:
gdf_viewer(tmp_for, un_val='ID')

Rows : 147, columns : 13, Unique on 'ID': 147


interactive(children=(IntSlider(value=10, description='rows', max=147, min=10, readout=False), IntSlider(value…

In [154]:
tmp_for.to_csv('../../CF_data/synthese/Result_traitem/boreholes_8.csv', index=False)

* **Feuille : 'RESULTS_SOL' (F)**

In [155]:
tmp_for = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Doc_SITEREM/Rapport de synthèse des études de sol et des eaux souterraines. Siterem - 2017/Database MEMORIS3.xlsx', sheet_name='RESULTS_SOL', skiprows=1)
gdf_viewer(tmp_for)

Rows : 1931, columns : 172


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:308: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


interactive(children=(IntSlider(value=10, description='rows', max=1931, min=10, readout=False), IntSlider(valu…

In [156]:
tmp_for.rename(columns={'Campagne':'Societe','N_forage':'ID','Prof.\nforage':'Long','refus':'Refus'}, inplace=True)
tmp_for=tmp_for[['ID','X','Y','Z','Long','Refus','Societe']][:1422]
tmp_for.replace('-',np.nan, inplace=True)
tmp_for['Refus']=tmp_for['Refus'].apply(lambda x: not pd.isnull(x))
tmp_for.drop_duplicates('ID', inplace=True)
tmp_for.reset_index(inplace=True, drop=True)

In [157]:
gdf_viewer(tmp_for, un_val='ID')

Rows : 330, columns : 7, Unique on 'ID': 330


interactive(children=(IntSlider(value=10, description='rows', max=330, min=10, readout=False), IntSlider(value…

In [158]:
tmp_for.to_csv('../../CF_data/synthese/Result_traitem/boreholes_9.csv', index=False)

### A.5- ../Data_UMONS/geometrie_electrodes_et_sondes/Forages_Pilote_Decoupe.xlsx

* **Feuille : 'leve' (F)**

In [159]:
For_synth=gpd.GeoDataFrame({})

In [160]:
tmp_for = pd.read_excel('../../CF_data/Data_UMONS/geometrie_electrodes_et_sondes/Forages_Pilote_Decoupe.xlsx', sheet_name='leve')#, skiprows=2)
gdf_viewer(tmp_for)

Rows : 72, columns : 12


interactive(children=(IntSlider(value=10, description='rows', max=72, min=10, readout=False), IntSlider(value=…

In [161]:
tmp_for.columns

Index(['Ref_puits', 'Niveau mesuré', 'Niveau corrigé', 'Bouteille',
       'decoupage [m]', 'Z_diff [m] repere_local', 'long_fin [m]',
       'Pos_Inox_#1 [m]', 'Pos_Inox_#6 [m]', 'Pos_Impol_#3 [m]', 'Unnamed: 10',
       'Unnamed: 11'],
      dtype='object')

In [162]:
tmp_for.rename(columns={'Ref_puits':'ID', 'Niveau corrigé':'Z','Z_diff [m] repere_local':'Diff_Z_Rloc', 
                        'long_fin [m]':'Long', 'Pos_Inox_#6 [m]':'Inox_first', 
                        'Pos_Impol_#3 [m]':'impol_first'}, inplace=True)
tmp_for=tmp_for[['ID','Z','Diff_Z_Rloc','Long','Inox_first', 'impol_first']]
#tmp_for.replace('-',np.nan, inplace=True)

In [163]:
gdf_viewer(tmp_for, un_val='ID') #

Rows : 72, columns : 6, Unique on 'ID': 72


interactive(children=(IntSlider(value=10, description='rows', max=72, min=10, readout=False), IntSlider(value=…

In [164]:
tmp_for.to_csv('../../CF_data/synthese/Result_traitem/boreholes_Leve_Z_Pilote.csv', index=False)

pour la suite extraire les fichiers avec XY

/////////////////////////////----- Brouillon de codes -----//////////////////////////////////////////////////////////////